# Deep Products: Building and Shipping Apps Using Deep Learning

This book attempts to simultaneously build and document the creation of a data-driven product using deep learning and natural language processing.

In [10]:
import sys, os, warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [11]:
import findspark
findspark.init()

import pyspark
def show(self, x=5):
    return self.limit(x).toPandas()
#pyspark.sql.DataFrame.show = show

import pyspark.sql.functions as F

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Deep Products').getOrCreate()

In [12]:
# Load patent base records, text summaries, citation links, CPC groups/subgroups and their names
patents = spark.read.csv('../data/patents*.csv.gz', header=True)
patents = patents.filter(patents.date > '1999-12-31')

patent_applications = spark.read.csv('../data/application.tsv.gz', header=True, sep='\t')

summaries = spark.read.csv('../data/summary*.csv.gz', header=True).drop('uuid')
summaries = summaries.withColumnRenamed('text', 'summary')

citations = spark.read.csv('../data/citation*.csv.gz', header=True).drop('uuid')
cpcs = spark.read.csv('../data/cpc*.csv.gz', header=True).drop('uuid')
cpc_groups = spark.read.csv('../data/cpc_group.tsv.gz', header=True, sep='\t')
cpc_subgroups = spark.read.csv('../data/cpc_subgroup.tsv.gz', header=True, sep='\t')

assignees = spark.read.csv('../data/assignee.tsv.gz', header=True, sep='\t')
patent_assignees = spark.read.csv('../data/patent_assignee.tsv.gz', header=True, sep='\t')
patent_inventors = spark.read.csv('../data/patent_inventor.tsv.gz', header=True, sep='\t')
inventors = spark.read.csv('../data/inventor.tsv.gz', header=True, sep='\t')

## Verify that the Application ID in the XML Applications Matches the Patentview Records

It does match, XML file patent application number `14380506` has a corresponding record in the patentview database with the same date, etc.

In [264]:
patent_applications.registerTempTable('patent_applications')
spark.sql('SELECT * FROM patent_applications WHERE number=="14380506"').show()

,id,patent_id,series_code,number,country,date
0,14/380506,9339069,14,14380506,US,2013-02-22


## Load the Patent Applications from the USPTO

Files from USPTO here: [https://developer.uspto.gov/product/patent-application-full-text-dataxml](https://developer.uspto.gov/product/patent-application-full-text-dataxml)

In [13]:
applications = spark.read.json('../data/applications/json/*.jsonl.gz')
applications.show()

+--------------------+--------------+--------------------+--------------------+
|            abstract|application_id|         description|               title|
+--------------------+--------------+--------------------+--------------------+
|Thermal managemen...|      16147573|description="Summ...|Reward-Based Upda...|
|An automotive tes...|      16076741|description="Cros...|Automotive Testin...|
|Compositions for ...|      15999302|description="Cros...|COMPOSITIONS FOR ...|
|An embodiment of ...|      16003219|description="Summ...|REDUCTION OF WRIT...|
|A door check mech...|      15538740|description="Summ...|DOOR CHECK MECHAN...|
|To provide a semi...|      16016182|description="Summ...|METHOD OF MANUFAC...|
|This application ...|      16154692|description="Cros...|METHOD AND APPARA...|
|The cosmetic prod...|      16073064|description="Summ...|Cosmetic Article ...|
|Power dissipation...|      15842311|description="Summ...|TOGGLE RATE REDUC...|
|The present inven...|      16076972|des

## Join the USPTO XML Applications to the Patentview Grant Data

Note that we limit because it kept crashing for out of disk space...

In [14]:
pas = applications.limit(50000).join(patent_applications, applications.application_id == patent_applications.number, 'left')
pas.registerTempTable('pas')

## Sanity check what proportion of patent applications were granted

In [21]:
spark.sql("""
SELECT 
    INT(ISNULL(patent_id)) AS granted,
    COUNT(*) AS total
FROM pas 
GROUP BY granted
""").show()

+-------+-----+
|granted|total|
+-------+-----+
|      1|45827|
|      0| 4185|
+-------+-----+



## Sanity check the grant counts by year

In [ ]:
spark.sql("""
SELECT 
    YEAR(date) as year, 
    INT(ISNULL(patent_id)) AS granted, 
    COUNT(*) as total
FROM pas
WHERE YEAR(date) > "2008-01-01"
GROUP BY 
    YEAR(date), 
    granted
ORDER BY 
    YEAR(date), 
    granted
""").show(20)

## Inspect the final records...

In [ ]:
spark.sql("""
SELECT 
    application_id, 
    title, 
    abstract, 
    description, 
    INT(ISNULL(patent_id)) AS granted 
FROM pas 
LIMIT 20
""").show()

## Write the training data out to disk as jsonl.gz files :)

In [17]:
final_pas = spark.sql("""SELECT application_id, title, abstract, description, INT(ISNULL(patent_id)) AS granted FROM pas""")
final_pas.write.json('../data/patent_applications/2019-04-06.jsonl.gz', compression='gzip')